In [1]:
!pip install rumbledb
%load_ext rumbledb
%env RUMBLEDB_SERVER=http://localhost:8002/jsoniq


env: RUMBLEDB_SERVER=http://localhost:8002/jsoniq


In [2]:
%%jsoniq
{ "pi" : 3.14, "sq2" : 1.4 }

Took: 1.2943952083587646 ms
{"pi": 3.14, "sq2": 1.4}


In [3]:
%%jsoniq
count(json-file("wasbs://newcontainer@rumbledb.blob.core.windows.net/git-archive.json"))


Took: 31.504942178726196 ms
36577


In [4]:
%%jsoniq
{ concat("Integer ", 2) : 2 * 2 }

Took: 0.18807125091552734 ms
{"Integer 2": 4}


### Question 1

Which actor commited the most events of the type "PushEvent", and how many did they commit?

In [7]:
%%jsoniq
for $git in json-file("wasbs://newcontainer@rumbledb.blob.core.windows.net/git-archive-big.json")
let $actor := $git.actor.id
group by $actor
where $git.type = "PushEvent"
let $cnt := count($git)
order by $cnt descending
count $c
where $c le 5
return {
"id" : $actor,
"count" : $cnt
}

Took: 35.78762078285217 ms
{"id": 56100, "count": 3843}
{"id": 280212, "count": 1832}
{"id": 2033581, "count": 966}
{"id": 1822073, "count": 720}
{"id": 9201970, "count": 673}


In [11]:
%%jsoniq
let $path := "wasbs://newcontainer@rumbledb.blob.core.windows.net/git-archive-big.json"
for $event in json-file($path)
group by $actor := $event.actor.id
where $event.type = "PushEvent"
let $count := count($event)
order by $count descending
count $c
where $c le 5
return {
"id" : $actor,
"count" : $count
}

Took: 27.963408946990967 ms
{"id": 56100, "count": 3843}
{"id": 280212, "count": 1832}
{"id": 2033581, "count": 966}
{"id": 1822073, "count": 720}
{"id": 9201970, "count": 673}


### Question 2

Which actor has committed the largest set of distinct event types?

In [14]:
%%jsoniq
let $path := "wasbs://newcontainer@rumbledb.blob.core.windows.net/git-archive-big.json"
for $event in json-file($path)
group by $actor := $event.actor.id
let $count := count(distinct-values($event.type))
order by $count descending
count $c
where $c le 5
return {
    "id" : $actor,
    "count" : $count
}

Took: 39.16199207305908 ms
{"id": 419884, "count": 11}
{"id": 756669, "count": 10}
{"id": 2829600, "count": 10}
{"id": 305268, "count": 9}
{"id": 677518, "count": 9}


### Question 3

At what time was the most recent fork of the repository "bitcoin/bitcoin" created?

In [18]:
%%jsoniq
let $path := "wasbs://newcontainer@rumbledb.blob.core.windows.net/git-archive-big.json"
for $event in json-file($path)
where $event.repo.name = "bitcoin/bitcoin" and $event.type = "ForkEvent"
order by $event.created_at descending
count $c
where $c le 5
return {
    "name" : $event.repo.name,
    "time" : $event.created_at,
}

Took: 29.562126874923706 ms
{"name": "bitcoin/bitcoin", "time": "2015-01-01T23:35:35Z", "type": "ForkEvent"}
{"name": "bitcoin/bitcoin", "time": "2015-01-01T22:54:30Z", "type": "ForkEvent"}
{"name": "bitcoin/bitcoin", "time": "2015-01-01T19:01:34Z", "type": "ForkEvent"}
{"name": "bitcoin/bitcoin", "time": "2015-01-01T15:51:15Z", "type": "ForkEvent"}
{"name": "bitcoin/bitcoin", "time": "2015-01-01T07:10:09Z", "type": "ForkEvent"}
